In [ ]:
# generate intron annotations from gtf

In [1]:
import getopt, re, os, sys, logging, time, datetime, copy
from collections import defaultdict

In [2]:
# smg-2 and N2
path = '/Volumes/DATA_2/sams'
gtf = 'collapsed_transcripts_N2_KH1668.gff'
strand = 'stranded'

In [8]:
# for filtered transcripts
path = '/Volumes/DATA_2/sams'
gtf = 'collapsed_transcripts_N2_KH1668.filtered.supported.named.gff'
strand = 'stranded'

In [2]:
# smg-2 and N2 L1
path = '/Volumes/DATA_2/sams'
gtf = 'collapsed_transcripts_N2_KH1668.L1.gff'
strand = 'stranded'

In [2]:
# N2 only
path = '/Volumes/DATA_3/GridION_N2_Nathan'
gtf = 'collapsed_transcripts_N2.gff'
strand = 'stranded'

In [8]:
# smg-2 only
path = '/Volumes/DATA_2/sams'
gtf = 'collapsed_transcripts_KH1668.gff'
strand = 'stranded'

In [27]:
# WS271
path = '/Users/kuroyanagi/expression/Caenorhabditis_elegans/WS271'
gtf = 'caenorhabditis_elegans.PRJNA13758.WBPS14.canonical_geneset.chr.gtf'
strand = 'stranded'

In [9]:
# use the awk command to generate Exon.gtf file
cmd = "less %s/%s | awk '{if($3==\"exon\"){print $0}}' > %s/Exon_%s" % (path, gtf, path, gtf)
os.system(cmd)     

0

In [4]:
# load file
def read_file(filename):
    with open(filename) as fp:
        List = [x.strip() for x in fp if len(x.strip()) > 0]
        return List

In [5]:
# gtf to introns
def intron_gtf(gtf_file_list):
    exon_s, exon_e = defaultdict(lambda: []), defaultdict(lambda: [])
    parse_line_start = 0
    for i in range(0, len(gtf_file_list)):
        if gtf_file_list[i].startswith('#') or gtf_file_list[i].split('\t')[2] != 'exon':
            continue
        parse_line_start = i
        break
    for i in range(parse_line_start, len(gtf_file_list)):
        gtf_list = gtf_file_list[i].strip().split('\t')
        if gtf_list[2] != 'exon':
            continue
        transcript_id = re.sub('.*transcript_id |;.*', '', gtf_list[8])
        exon_s[transcript_id].append(int(gtf_list[3]))
        exon_e[transcript_id].append(int(gtf_list[4]))
    trans = {}
    fw = open(("%s/Intron_%s" % (path, gtf)), "w")
    for i in range(parse_line_start, len(gtf_file_list)):
        gtf_list = gtf_file_list[i].strip().split('\t')
        transcript_id = re.sub('.*transcript_id |;.*', '', gtf_list[8])
        gene_id = re.sub('.*gene_id |;.*', '', gtf_list[8])
        if transcript_id in trans:
            trans[transcript_id] = 'false'
        elif len(exon_s[transcript_id]) > 1:
            exon_s[transcript_id].sort()
            exon_e[transcript_id].sort()
            for i in range(1, len(exon_s[transcript_id])):
                if gtf_list[6] == '+':
                    fw.write(
                        "%s\t%s\tintron\t%s\t%s\t.\t%s\t.\tgene_id %s; transcript_id %s; intron_number %s; total_intron_number %s; exon_1 %s_%s; exon_2 %s_%s;\n" % (
                            gtf_list[0], gtf_list[1], str(int(exon_e[transcript_id][i - 1]) + 1),
                            str(int(exon_s[transcript_id][i]) - 1),
                            gtf_list[6], gene_id, transcript_id, i, len(exon_s[transcript_id]) - 1,
                            exon_s[transcript_id][i - 1], exon_e[transcript_id][i - 1], exon_s[transcript_id][i],
                            exon_e[transcript_id][i]))
                else:
                    fw.write(
                        "%s\t%s\tintron\t%s\t%s\t.\t%s\t.\tgene_id %s; transcript_id %s; intron_number %s; total_intron_number %s; exon_1 %s_%s; exon_2 %s_%s;\n" % (
                            gtf_list[0], gtf_list[1], str(int(exon_e[transcript_id][i - 1]) + 1),
                            str(int(exon_s[transcript_id][i]) - 1),
                            gtf_list[6], gene_id, transcript_id, len(exon_s[transcript_id]) - i,
                            len(exon_s[transcript_id]) - 1, exon_s[transcript_id][i - 1], exon_e[transcript_id][i - 1],
                            exon_s[transcript_id][i], exon_e[transcript_id][i]))
        trans[transcript_id] = 'true'
    fw.close()

In [10]:
gtf_file_list = read_file('%s/%s' % (path, gtf))

In [11]:
intron_gtf(gtf_file_list)